In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.2+cu118
    Uninstalling torchaudio-2.0.2+cu118:
      Successfully uninstalled torchaudio-2.0.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sou

In [ ]:
!pip install pytorch_lightning==1.9.4
!pip install pytorch_forecasting==0.10.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 14.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.

In [ ]:
import sys
import os
import argparse
import shutil
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.metrics import QuantileLoss

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor
)
from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


In [ ]:
# category columns
CATE_COLS = ['B', 'M', 'S', 'BR', 'ID','P']
#CATE_COLS = ['ID']

ENCODER_LENGTH_IN_WEEKS = 60

# learning rate determined by a cv run with train data less 1 trailing week as validation
LRS = [0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306 , 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307,
       0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307]

# number of epochs found in cv run
NUM_EPOCHS = 60

# number of seeds to use
NUM_SEEDS = 10

BATCH_SIZE = 4  #

# hyper parameters determined by cv runs with train data less 1 trailing week as validation
PARAMS = {
    'gradient_clip_val': 0.9658579636307634,
    'hidden_size': 10, #180
    'dropout': 0.19610151695402608,
    'hidden_continuous_size': 5, #90
    'attention_head_size': 4,
    'learning_rate': 0.08
}

In [ ]:
#경로만 자신의 환경에 맞게 잘 설정해주세요!
DATAROOT='/content/drive/MyDrive/Aimers3'
CKPTROOT = DATAROOT+"/ckpts" # directory for model checkpoints
CSVROOT = DATAROOT+"/csvs" # directory for prediction outputs
SUBFN = DATAROOT+"/sub.csv" # final submission file path
LOGDIR = DATAROOT+"/logs" # pytorch_forecasting requirs logger

In [ ]:
cnt = pd.read_csv('/content/drive/MyDrive/Aimers3/brand_keyword_cnt.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/Aimers3/sample_submission.csv')
train = pd.read_csv('/content/drive/MyDrive/Aimers3/train.csv')
sales = pd.read_csv('/content/drive/MyDrive/Aimers3/sales.csv')
info = pd.read_csv('/content/drive/MyDrive/Aimers3/product_info.csv')

In [ ]:
train = train[(train['ID'] == 224) | (train['ID'] == 225)]
train

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
224,224,B002-00071-00005,B002-C001-0002,B002-C002-0007,B002-C003-0035,B002-00071,4,3,1,3,...,15,12,9,24,14,34,12,18,59,47
225,225,B002-00071-00006,B002-C001-0002,B002-C002-0003,B002-C003-0016,B002-00071,0,0,0,0,...,0,0,0,25,24,9,3,6,15,43


In [ ]:
date_columns = submission_df.columns[1:]
submission_df = submission_df.melt(id_vars=['ID'],
                             value_vars=date_columns,
                             var_name='날짜',
                             value_name='값')
submission_df['날짜'] = pd.to_datetime(submission_df['날짜'])
submission_df

,ID,날짜,값
0,0,2023-04-05,0
1,1,2023-04-05,0
2,2,2023-04-05,0
3,3,2023-04-05,0
4,4,2023-04-05,0
...,...,...,...
333685,15885,2023-04-25,0
333686,15886,2023-04-25,0
333687,15887,2023-04-25,0
333688,15888,2023-04-25,0


In [ ]:
date_columns = train.columns[6:]
train = train.melt(id_vars=['ID', '제품', '대분류', '중분류', '소분류', '브랜드'],
                             value_vars=date_columns,
                             var_name='날짜',
                             value_name='값')
train['날짜'] = pd.to_datetime(train['날짜'])
train

,ID,제품,대분류,중분류,소분류,브랜드,날짜,값
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022-01-01,0
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,2022-01-01,0
...,...,...,...,...,...,...,...,...
7293505,15885,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,2023-04-04,0
7293506,15886,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,3
7293507,15887,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,0
7293508,15888,B002-03799-00005,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,2


In [ ]:
brand_keyword_long_format = cnt.melt(id_vars=['브랜드'],
                                                    value_vars=date_columns,
                                                    var_name='날짜',
                                                    value_name='키워드_값')
brand_keyword_long_format['날짜'] = pd.to_datetime(brand_keyword_long_format['날짜'])
train = pd.merge(train, brand_keyword_long_format, how='left', on=['브랜드', '날짜'])
train.head()

,ID,제품,대분류,중분류,소분류,브랜드,날짜,값,키워드_값
0,224,B002-00071-00005,B002-C001-0002,B002-C002-0007,B002-C003-0035,B002-00071,2022-01-01,4,1.015361
1,225,B002-00071-00006,B002-C001-0002,B002-C002-0003,B002-C003-0016,B002-00071,2022-01-01,0,1.015361
2,224,B002-00071-00005,B002-C001-0002,B002-C002-0007,B002-C003-0035,B002-00071,2022-01-02,3,3.684406
3,225,B002-00071-00006,B002-C001-0002,B002-C002-0003,B002-C003-0016,B002-00071,2022-01-02,0,3.684406
4,224,B002-00071-00005,B002-C001-0002,B002-C002-0007,B002-C003-0035,B002-00071,2022-01-03,1,3.336253


In [ ]:
train['날짜'] = pd.to_datetime(train['날짜'])
train['dow'] = train['날짜'].dt.weekday
#train['date'] = train['날짜'].dt.date.astype('str')
train['day'] = train['날짜'].dt.day
train['month'] = train['날짜'].dt.month
train

,ID,제품,대분류,중분류,소분류,브랜드,날짜,값,dow,day,month
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022-01-01,0,5,1,1
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0,5,1,1
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0,5,1,1
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0,5,1,1
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,2022-01-01,0,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...
7293505,15885,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,2023-04-04,0,1,4,4
7293506,15886,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,3,1,4,4
7293507,15887,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,0,1,4,4
7293508,15888,B002-03799-00005,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,2,1,4,4


In [ ]:
train = train.sort_values(['ID','날짜'])
train = train.reset_index(drop = True)
train.drop(['제품','대분류','중분류','소분류','브랜드'],axis=1,inplace=True)
train = train.fillna(0)
train

,ID,날짜,값,dow,day,month
0,0,2022-01-01,0,5,1,1
1,0,2022-01-02,0,6,2,1
2,0,2022-01-03,0,0,3,1
3,0,2022-01-04,0,1,4,1
4,0,2022-01-05,0,2,5,1
...,...,...,...,...,...,...
7293505,15889,2023-03-31,0,4,31,3
7293506,15889,2023-04-01,0,5,1,4
7293507,15889,2023-04-02,0,6,2,4
7293508,15889,2023-04-03,0,0,3,4


In [ ]:
submission_df['ID'] = submission_df['ID'].astype(int)
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333690 entries, 0 to 333689
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   ID      333690 non-null  int64         
 1   날짜      333690 non-null  datetime64[ns]
 2   값       333690 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 7.6 MB


In [ ]:
submission_df = submission_df.sort_values(['ID','날짜'])
submission_df = submission_df.reset_index(drop = True)
submission_df

,ID,날짜,값
0,0,2023-04-05,0
1,0,2023-04-06,0
2,0,2023-04-07,0
3,0,2023-04-08,0
4,0,2023-04-09,0
...,...,...,...
333685,15889,2023-04-21,0
333686,15889,2023-04-22,0
333687,15889,2023-04-23,0
333688,15889,2023-04-24,0


In [ ]:
'''
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드','제품']

for col in categorical_columns:
    label_encoder.fit(train[col])
    train[col] = label_encoder.transform(train[col])

train['대분류'] = train['대분류'].astype(str)

train['중분류'] = train['중분류'].astype(str)

train['소분류'] = train['소분류'].astype(str)

train['브랜드'] = train['브랜드'].astype(str)

train['제품'] = train['제품'].astype(str)

train = train.fillna(0)
train
'''

In [ ]:
# 누락된 값을 'Unknown'으로 대체
#train.fillna('Unknown', inplace=True)
#submission_df.fillna('Unknown', inplace=True)

# ID 열을 문자열로 변환
train['ID'] = train['ID'].astype(str)
submission_df['ID'] = submission_df['ID'].astype(str)

In [ ]:
CATE_COLS = ['ID', 'dow', 'day', 'month']

In [ ]:
def seed_all(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
[os.makedirs(p, exist_ok=True) for p in (CKPTROOT, CSVROOT, LOGDIR)]

[None, None, None]

In [ ]:
train['dow'] = train['dow'].astype(str)

train['day'] = train['day'].astype(str)

train['month'] = train['month'].astype(str)

KeyError: ignored

In [ ]:
from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)

In [ ]:
train = train.fillna(0)

In [ ]:
__ix = train.columns.get_loc('날짜')
train['날짜'] = (train.loc[:, '날짜'] - train.iloc[0, __ix]).astype('timedelta64[D]').astype('int')
train

,ID,날짜,값,dow,day,month
0,0,0,0,5,1,1
1,0,1,0,6,2,1
2,0,2,0,0,3,1
3,0,3,0,1,4,1
4,0,4,0,2,5,1
...,...,...,...,...,...,...
7293505,15889,454,0,4,31,3
7293506,15889,455,0,5,1,4
7293507,15889,456,0,6,2,4
7293508,15889,457,0,0,3,4


In [ ]:
train.rename(columns={
    '키워드_값' : 'key',
    '값' : 'target',
    '날짜' : 'time_idx'
}, inplace=True)
train

,ID,time_idx,target,dow,day,month
0,0,0,0,5,1,1
1,0,1,0,6,2,1
2,0,2,0,0,3,1
3,0,3,0,1,4,1
4,0,4,0,2,5,1
...,...,...,...,...,...,...
7293505,15889,454,0,4,31,3
7293506,15889,455,0,5,1,4
7293507,15889,456,0,6,2,4
7293508,15889,457,0,0,3,4


In [ ]:
train["log_target"] = np.log(train.target + 1e-8)
train["avg_target_by_ID"] = train.groupby(["time_idx", "ID"], observed=True).target.transform("mean")
#train["avg_volume_by_agency"] = train.groupby(["time_idx", "agency"], observed=True).volume.transform("mean")

In [ ]:
# window size 설정
'''
window_size = 30

# 각 ID별로 groupby
grouped = train.groupby('ID')

# 이동 평균 계산
def apply_moving_average(group):
    group['target'] = group['target'].rolling(window=window_size, min_periods=1).mean()
    # null 값은 원래 값으로 대체
    group['target'].fillna(group['target'], inplace=True)
    return group

train = grouped.apply(apply_moving_average)
train
'''

In [ ]:
'''
train.rename(columns={
    '대분류': 'B',
    '중분류': 'M',
    '소분류': 'S',
    '제품': 'P',
    '브랜드': 'BR',
    '키워드_값' : 'key',
    '값' : 'target',
    '날짜' : 'time_idx'
}, inplace=True)
train
'''

In [ ]:
#train.drop(['키워드_값'],axis=1,inplace=True)

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import torch

from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import MAE, SMAPE, MultivariateNormalDistributionLoss

In [ ]:
train.drop(['dow','day','month'],axis=1,inplace=True)

In [ ]:
def load_dataset(train_df, validate=False):
    max_encoder_length = 60 #5
    max_prediction_length = 21
    context_length = max_encoder_length
    prediction_length = max_prediction_length
    #train_df['날짜'] = (train_df['날짜'] - train_df['날짜'].min()).dt.days
    train_df['target'] = train_df['target'].astype(float)
    train_df['ID'] = train_df['ID'].astype(str) # 문자열로 변환

    training_cutoff = train_df['time_idx'].max()-max_prediction_length #2040 - 24*7 = 1871

    tr_ds = TimeSeriesDataSet(
    train_df[lambda x: x.time_idx <= training_cutoff],
    #train_df,
    time_idx="time_idx",
    target="target",
    categorical_encoders={"ID": NaNLabelEncoder().fit(train_df.ID)},
    group_ids=["ID"],
    static_categoricals=[
        "ID"
    ],  # as we plan to forecast correlations, it is important to use series characteristics (e.g. a series identifier)
    time_varying_unknown_reals=["target"],
    #min_encoder_length = context_length // 2,       #1,
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,
    )
    va_ds = None
    if validate:
        va_ds = TimeSeriesDataSet.from_dataset(
        tr_ds, train_df, predict=True, stop_randomization=True
    )

    return tr_ds, va_ds

In [ ]:
tr_ds, va_ds = load_dataset(train, validate=False)
#tr_ds, va_ds = load_dataset(train_df, validate=True)
tr_ds

/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/data/encoders.py:899: UserWarning: scale is below 1e-7 - consider not centering the data or using data with higher variance for numerical stability
  warnings.warn(


TimeSeriesDataSet[length=5688620](
	time_idx='time_idx',
	target='target',
	group_ids=['ID'],
	weight=None,
	max_encoder_length=60,
	min_encoder_length=60,
	min_prediction_idx=0,
	min_prediction_length=21,
	max_prediction_length=21,
	static_categoricals=['ID'],
	static_reals=[],
	time_varying_known_categoricals=[],
	time_varying_known_reals=[],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['target'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation=None,
	method_kwargs={}
),
	categorical_encoders={'ID': NaNLabelEncoder(add_nan=False, warn=True), '__group_id__ID': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={},
	randomize_length=None,
	predict_mode=False
)

In [ ]:
train

,ID,time_idx,target
0,224,0,4.0
1,224,1,3.0
2,224,2,1.0
3,224,3,3.0
4,224,4,3.0
...,...,...,...
913,225,454,9.0
914,225,455,3.0
915,225,456,6.0
916,225,457,15.0


In [ ]:
from pytorch_forecasting.data import TimeSeriesDataSet
from torch.utils.data import DataLoader

# 데이터셋 로드
tr_ds, _ = load_dataset(train, validate=False)

In [ ]:
from pytorch_forecasting.data import TimeSeriesDataSet
from torch.utils.data import DataLoader
train_loader = DataLoader(tr_ds, batch_size=830)  # 배치 크기는 필요에 따라 조정
train_loader

In [ ]:
from torchmetrics import Metric
import torch

class PSFA(Metric):
    def __init__(self, group_ids):
        super().__init__()
        self.group_ids = group_ids
        self.add_state("sum_psfa_m", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds, target):
        # 일일 예측과 실제 값
        y_day = target
        p_day = preds

        # 대분류 별로 루프를 돌며 계산
        for m in self.group_ids:
            # m 대분류에 해당하는 행만 선택
            y_m = y_day[m]
            p_m = p_day[m]

            # 대분류 별 Pseudo 예측 정확도를 계산
            psfa_m = 1 - (torch.abs(y_m - p_m) / torch.max(y_m, p_m) * y_m / y_m.sum()).sum() / y_m.size(0)

            # 총합을 업데이트
            self.sum_psfa_m += psfa_m
            self.total += 1

    def compute(self):
        # 전체 Pseudo 예측 정확도를 계산
        return self.sum_psfa_m / self.total
# 대분류 식별자 목록을 생성
#group_ids = train['B'].unique()
group_ids = 5

# 사용자 정의 메트릭 인스턴스를 생성
custom_psfa = PSFA(group_ids)

In [ ]:
#early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
tr_loader = tr_ds.to_dataloader(train=True, batch_size=BATCH_SIZE, num_workers=12)
trainer = pl.Trainer(
    max_epochs=15,
    accelerator="gpu",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    #callbacks=[early_stop_callback],
    limit_train_batches=50,
    enable_checkpointing=True,
)

net = DeepAR.from_dataset(
    dataset=tr_ds,
    learning_rate=1e-2,
    hidden_size=30,
    rnn_layers=2,
    loss=MultivariateNormalDistributionLoss(rank=3),
)




trainer.fit(
    net,
    train_dataloaders=tr_loader,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                   | Type                               | Params
------------------------------------------------------------------------------
0 | loss                   | MultivariateNormalDistributionLoss | 0     
1 | logging_metrics        | ModuleList                         | 0     
2 | embeddings             | MultiEmbedding                     | 1.6 M 
3 | rnn                    | LSTM                               | 23.4 K
4 | distribution_projector | Linear                             | 155   
----------------------------

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


In [ ]:
# training
def fit(seed, tr_ds, va_loader=None):
    seed_all(seed) # doesn't really work as training is non-deterministic

    # create dataloaders for model
    tr_loader = tr_ds.to_dataloader(
        train=True, batch_size=BATCH_SIZE, num_workers=12
    )

    if va_loader is not None:
        # stop training, when loss metric does not improve on validation set
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            min_delta=1e-4,
            patience=20,
            verbose=True,
            mode="min"
        )
        lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
        callbacks = [lr_logger, early_stopping_callback]
    else:
        # gather 10 checkpoints with best traing loss
        checkpoint_callback = ModelCheckpoint(
            monitor='train_loss',
            dirpath=CKPTROOT,
            filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
            save_top_k=3 #10
        )
        callbacks = [checkpoint_callback]
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    trainer = pl.Trainer(
        max_epochs=30,
        accelerator="gpu",
        enable_model_summary=True,
        gradient_clip_val=0.1,
        callbacks=[early_stop_callback],
        limit_train_batches=50,
        enable_checkpointing=True,
    )


    net = DeepAR.from_dataset(
        tr_ds,
        learning_rate=1e-2,
        log_interval=10,
        log_val_interval=1,
        hidden_size=30,
        rnn_layers=2,
        optimizer="Adam",
        loss=MultivariateNormalDistributionLoss(rank=30),
    )

    print(f"Number of parameters in network: {net.size()/1e3:.1f}k")

    kwargs = {'train_dataloaders': tr_loader}
    if va_loader:
        kwargs['val_dataloaders'] = va_loader

    # fit network
    trainer.fit(
        net,
        **kwargs
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    print(f'best_model_path={best_model_path}')
    best_tft = DeepAR.load_from_checkpoint(best_model_path)

    return best_tft

In [ ]:
#seed=[17, 1218, 20230725, 1998, 32, 40, 800, 6651, 4931, 18011810]

seed=[19990313]
for s in seed:
    fit(s, tr_ds)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


AssertionError: ignored

In [ ]:
tr_ds[0]

In [ ]:
tr_ds[53]

In [ ]:
def forecast(ckpt, train_df,tr_ds):
    # 모델 로드
    best_tft = DeepAR.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = 21
    print(max_encoder_length)
    # 마지막 5주 데이터 사용
    encoder_data = train_df[lambda x: x.time_idx > x.time_idx.max() - 81]

    # 마지막 행 추출
    last_data = train_df.iloc[[-1]]
    #encoder_data = encoder_data[encoder_data['ID'] == "224"]
    # 테스트 데이터 준비 (21일 예측)
    #decoder_data = train_df.iloc[[-1]].copy()
    #print(decoder_data)
    #decoder_data['time_idx'] += np.arange(1, max_prediction_length+1)

    # 예측을 위한 데이터 병합
    #new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_prediction_data = encoder_data.copy()
    print(new_prediction_data)
    new_raw_predictions, _ = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    preds = new_raw_predictions['prediction'].squeeze().numpy()
    preds = np.round(preds, 0).astype(int)

    submission_df = pd.read_csv(DATAROOT+"/sample_submission.csv")
    train = pd.read_csv(DATAROOT+"/train.csv")

    date_columns = submission_df.columns[1:]
    submission_df = submission_df.melt(id_vars=['ID'],
                                value_vars=date_columns,
                                var_name='날짜',
                                value_name='값')
    submission_df['날짜'] = pd.to_datetime(submission_df['날짜'])
    submission_df = submission_df.sort_values(['ID','날짜'])
    submission_df = submission_df.reset_index(drop = True)

    #submission_df = submission_df.iloc[:10500,:]
    print(preds)
    submission_df['값'] = preds.flatten()
    #print(submission_df[submission_df['ID'] == 224])
    print(submission_df)
    original_format_df = submission_df.pivot(index='ID', columns='날짜', values='값').reset_index()

    original_format_df.columns = ['ID'] + [col.strftime('%Y-%m-%d') for col in original_format_df.columns[1:]]
    outfn = CSVROOT + '/' + (Path(ckpt).stem + '.csv')
    #print(original_format_df)
    original_format_df.to_csv(outfn, index=False)
    #interpretation = best_tft.interpret_output(new_raw_predictions.output, reduction="sum")
    #best_tft.plot_interpretation(interpretation)
    return preds
import glob
print("### FORECAST ###")
for p in glob.glob("/content/lightning_logs/version_1/checkpoints/epoch=14-step=750.ckpt"):
    preds = forecast(p, train,tr_ds)
    print(preds)
preds

### FORECAST ###


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


60
            ID  time_idx  target
378          0       378     0.0
379          0       379     4.0
380          0       380     0.0
381          0       381     1.0
382          0       382     2.0
...        ...       ...     ...
7293505  15889       454     0.0
7293506  15889       455     0.0
7293507  15889       456     0.0
7293508  15889       457     0.0
7293509  15889       458     0.0

[1287090 rows x 3 columns]


/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/data/encoders.py:899: UserWarning: scale is below 1e-7 - consider not centering the data or using data with higher variance for numerical stability
  warnings.warn(


[[[   0    2    2 ...    3    1    2]
  [  -2    0    2 ...    2    0   -3]
  [  -2    0    1 ...    4    1    1]
  ...
  [   2    0    3 ...    0   -1    2]
  [   1    0    2 ...    0   -2   -2]
  [   0   -1    1 ...    0    2    2]]

 [[   1    0    1 ...    1    1    0]
  [   1    0    2 ...    2    1   -1]
  [   0    0    1 ...    1    1   -1]
  ...
  [   1    0    4 ...    1    1   -1]
  [  -1    0    2 ...    0    0    1]
  [   0    1    0 ...    0    0    1]]

 [[   0    0    0 ...    0    0    0]
  [   0    0    0 ...    0    0    0]
  [   0    0    0 ...    0    0    0]
  ...
  [   0    0    0 ...    0    0    0]
  [   0    0    0 ...    0    0    0]
  [   0    0    0 ...    0    0    0]]

 ...

 [[  25   13 -136 ...  104  115   -5]
  [ -18    4  -67 ...   65  116 -101]
  [ -15  163  121 ...  -74   93   62]
  ...
  [ 160  -35   70 ...  183  185  -21]
  [ 236   18  214 ...   59  262   51]
  [ 183  -55  155 ...  104  223   40]]

 [[   0    0    0 ...    0    0    0]
  [   0    0

ValueError: ignored

In [ ]:
def forecast(ckpt, train_df,tr_ds):
    # 모델 로드
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = 21

    # 마지막 5주 데이터 사용
    encoder_data = train_df[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

    # 마지막 행 추출
    last_data = train_df.iloc[[-1]]

    # 테스트 데이터 준비 (21일 예측)
    #decoder_data = train_df.iloc[[-1]].copy()
    #print(decoder_data)
    #decoder_data['time_idx'] += np.arange(1, max_prediction_length+1)

    # 예측을 위한 데이터 병합
    #new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_prediction_data = encoder_data.copy()
    print(new_prediction_data)
    #print(new_prediction_data)
    #new_raw_predictions, _ = best_tft.predict(new_prediction_data, mode="raw", return_x=True)
    new_raw_predictions, _ = best_tft.predict(tr_ds.filter(lambda x: (x.ID == "ID") & (x.time_idx_first_prediction == 21)),
    mode="raw",
    return_x=True,)

    # 예측 결과 추출
    preds = new_raw_predictions['prediction'].squeeze().numpy()
    preds = np.round(preds, 0).astype(int)
    # 제출 형식에 맞게 변환
    # 제출 형식에 맞게 변환
    submission_df = pd.read_csv(DATAROOT+"/sample_submission.csv")
    train = pd.read_csv(DATAROOT+"/train.csv")

    #train = train[train['ID'] == 224]
    #common_ids = set(train['ID']).intersection(set(submission_df['ID']))

    #submission_df = submission_df[submission_df['ID'].isin(common_ids)]
    #submission_df = submission_df[(submission_df['ID'] == 0) | (submission_df['ID'] == 1)]
    date_columns = submission_df.columns[1:]
    submission_df = submission_df.melt(id_vars=['ID'],
                                value_vars=date_columns,
                                var_name='날짜',
                                value_name='값')
    submission_df['날짜'] = pd.to_datetime(submission_df['날짜'])
    submission_df = submission_df.sort_values(['ID','날짜'])
    submission_df = submission_df.reset_index(drop = True)

    #submission_df = submission_df.iloc[:10500,:]

    submission_df['값'] = preds.flatten()
    # 날짜 열을 피처로 변환하기 위해 pivot 함수 사용
    #print(submission_df)
    original_format_df = submission_df.pivot(index='ID', columns='날짜', values='값').reset_index()

    # 열 이름을 정확한 날짜 형식으로 변경
    original_format_df.columns = ['ID'] + [col.strftime('%Y-%m-%d') for col in original_format_df.columns[1:]]
    outfn = CSVROOT + '/' + (Path(ckpt).stem + '.csv')
    print(original_format_df)
    original_format_df.to_csv(outfn, index=False)
    #interpretation = best_tft.interpret_output(new_raw_predictions.output, reduction="sum")
    #best_tft.plot_interpretation(interpretation)
    return preds
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    preds = forecast(p, train)
    print(preds)
preds

### FORECAST ###


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


            ID  time_idx  target      key dow day month    log_target  \
94           0        94     1.0  1.23295   1   5     4  1.000000e-08   
95           0        95     0.0  0.62373   2   6     4 -1.842068e+01   
96           0        96     0.0  0.85581   3   7     4 -1.842068e+01   
97           0        97     0.0  0.49318   4   8     4 -1.842068e+01   
98           0        98     0.0  0.37713   5   9     4 -1.842068e+01   
...        ...       ...     ...      ...  ..  ..   ...           ...   
7293505  15889       454     0.0  5.51203   4  31     3 -1.842068e+01   
7293506  15889       455     0.0  3.52480   5   1     4 -1.842068e+01   
7293507  15889       456     0.0  4.03249   6   2     4 -1.842068e+01   
7293508  15889       457     0.0  5.88917   0   3     4 -1.842068e+01   
7293509  15889       458     0.0  5.07687   1   4     4 -1.842068e+01   

         avg_target_by_ID  
94                    1.0  
95                    0.0  
96                    0.0  
97         

AttributeError: ignored

# 여끼까지요

In [ ]:
# 제출 형식에 맞게 변환
submission_df = pd.read_csv(DATAROOT+"/sample_submission.csv")
#submission_df = submission_df[(submission_df['ID'] == 0) | (submission_df['ID'] == 1)]
date_columns = submission_df.columns[1:]
submission_df = submission_df.melt(id_vars=['ID'],
                            value_vars=date_columns,
                            var_name='날짜',
                            value_name='값')
submission_df['날짜'] = pd.to_datetime(submission_df['날짜'])
submission_df = submission_df.sort_values('ID')
submission_df['값'] = preds.flatten()
submission_df.describe()

ValueError: ignored

In [ ]:
# 날짜 열을 피처로 변환하기 위해 pivot 함수 사용
original_format_df = submission.pivot(index='ID', columns='날짜', values='값').reset_index()

# 열 이름을 정확한 날짜 형식으로 변경
original_format_df.columns = ['ID'] + [col.strftime('%Y-%m-%d') for col in original_format_df.columns[1:]]

# 결과 확인
original_format_df.head()

In [ ]:
from tqdm.notebook import tqdm

def forecast(ckpt, train_df):
    # load model
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = best_tft.dataset_parameters['max_prediction_length']

    assert max_encoder_length == 60 and max_prediction_length == 21
    '''
    # Initialize list to store each DataFrame
    prediction_data_list = []
    decoder_data_list = []

    # Wrap tqdm around the loop to display progress
    for ID in tqdm(train_df['ID'].unique(), desc="Processing IDs"):
        # use 5 weeks of training data at the end
        encoder_data = train_df[(train_df['ID'] == ID) & (train_df['날짜'] > train_df['날짜'].max() - max_encoder_length)]

        # get last entry from training data for this ID
        last_data = encoder_data.iloc[[-1]]

        # create dummy decoder data by replicating the last entry from training data
        decoder_data = pd.concat([last_data] * max_prediction_length, ignore_index=True)

        # modify the 날짜 for the decoder data
        decoder_data['날짜'] = range(last_data['날짜'].item() + 1, last_data['날짜'].item() + max_prediction_length + 1)

        # combine encoder and decoder data. decoder data is to be predicted
        id_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
        decoder_data_list.append(decoder_data)

        # Add to list
        prediction_data_list.append(id_prediction_data)
    all_decoder_data = pd.concat(decoder_data_list, ignore_index=True)

    # Save the decoder data to CSV
    all_decoder_data.to_csv('decoder_data.csv', index=False)
    # Concatenate all DataFrames in the list at once
    new_prediction_data = pd.concat(prediction_data_list, ignore_index=True)
    new_prediction_data.to_csv('new_prediction_data.csv',index=False)
    '''
    new_prediction_data = pd.read_csv('/content/new_prediction_data.csv')

    #print(new_prediction_data.head(30))
    #print("1111111111111111111")
    new_prediction_data['ID'] = new_prediction_data['ID'].astype(str)
    #print(new_prediction_data.info())
    new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)
    #print("2222222222222222222")


    num_labels = best_tft.dataset_parameters['categorical_encoders']['ID'].classes_

    # 예측 결과 추출
    preds = new_raw_predictions['prediction'].squeeze().numpy()

    # 제출 형식에 맞게 변환
    # 제출 형식에 맞게 변환
    submission_df = pd.read_csv(DATAROOT+"/sample_submission.csv")
    #submission_df = submission_df[(submission_df['ID'] == 0) | (submission_df['ID'] == 1)]
    date_columns = submission_df.columns[1:]
    submission_df = submission_df.melt(id_vars=['ID'],
                                value_vars=date_columns,
                                var_name='날짜',
                                value_name='값')
    submission_df['날짜'] = pd.to_datetime(submission_df['날짜'])
    submission_df = submission_df.sort_values('ID')
    submission_df['값'] = preds.flatten()
    # 날짜 열을 피처로 변환하기 위해 pivot 함수 사용
    original_format_df = submission_df.pivot(index='ID', columns='날짜', values='값').reset_index()

    # 열 이름을 정확한 날짜 형식으로 변경
    original_format_df.columns = ['ID'] + [col.strftime('%Y-%m-%d') for col in original_format_df.columns[1:]]
    outfn = CSVROOT + '/' + (Path(ckpt).stem + '.csv')
    print(original_format_df)
    original_format_df.to_csv(outfn, index=False)
    return preds
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    submission_df = forecast(p, train)
    print(submission_df)

In [ ]:
submission_df

In [ ]:
from tqdm.notebook import tqdm

def forecast(ckpt, train_df):
    # load model
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = best_tft.dataset_parameters['max_prediction_length']

    assert max_encoder_length == 60 and max_prediction_length == 21

    # Initialize list to store each DataFrame
    prediction_data_list = []
    decoder_data_list = []

    # Wrap tqdm around the loop to display progress
    for ID in tqdm(train_df['ID'].unique(), desc="Processing IDs"):
        # use 5 weeks of training data at the end
        encoder_data = train_df[(train_df['ID'] == ID) & (train_df['날짜'] > train_df['날짜'].max() - max_encoder_length)]

        # get last entry from training data for this ID
        last_data = encoder_data.iloc[[-1]]

        # create dummy decoder data by replicating the last entry from training data
        decoder_data = pd.concat([last_data] * max_prediction_length, ignore_index=True)

        # modify the 날짜 for the decoder data
        decoder_data['날짜'] = range(last_data['날짜'].item() + 1, last_data['날짜'].item() + max_prediction_length + 1)

        # combine encoder and decoder data. decoder data is to be predicted
        id_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
        decoder_data_list.append(decoder_data)

        # Add to list
        prediction_data_list.append(id_prediction_data)
    all_decoder_data = pd.concat(decoder_data_list, ignore_index=True)

    # Save the decoder data to CSV
    all_decoder_data.to_csv('decoder_data.csv', index=False)
    # Concatenate all DataFrames in the list at once
    new_prediction_data = pd.concat(prediction_data_list, ignore_index=True)
    new_prediction_data.to_csv('new_prediction_data.csv',index=False)

    print(last_data)
    print("1111111111111111111")
    new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)
    print("2222222222222222222")

    # num_labels: mapping from 'num' categorical feature to index in new_raw_predictions['prediction']
    #             {'5': 4, '6': 6, ...}
    # new_raw_predictions['prediction'].shape = (60, 168, 1)
    num_labels = best_tft.dataset_parameters['categorical_encoders']['ID'].classes_

    preds = new_raw_predictions['prediction'].squeeze()

    sub_df = pd.read_csv(DATAROOT + "/sample_submission.csv")

    # get prediction for each building (num)
    for n, ix in num_labels.items():
        sub_df.loc[(sub_df['ID'].apply(lambda x: int(x.split('_')[0])) == int(n)), '값'] = preds[ix].numpy()
    for n, ix in num_labels.items():
        sub_df.loc[sub_df['ID'] == int(n), '값'] = preds[ix].numpy()

    # save prediction to a CSV file
    outfn = CSVROOT + '/' + (Path(ckpt).stem + '.csv')
    print(outfn)
    sub_df.to_csv(outfn, index=False)
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    submission_df = forecast(p, train)


In [ ]:
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    submission_df = forecast(p, train)


In [ ]:
def forecast(ckpt, train_df):
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = 21

    # 각 ID별로 마지막 max_encoder_length 데이터 사용
    encoder_data_list = []
    for ID in train_df['ID'].unique():
        encoder_data_per_id = train_df[train_df['ID'] == ID].iloc[-max_encoder_length:]
        encoder_data_list.append(encoder_data_per_id)
    encoder_data = pd.concat(encoder_data_list, ignore_index=True)


    # 예측을 위한 데이터 병합
    #new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_prediction_data = encoder_data.copy()
    # 예측 수행
    new_raw_predictions, _ = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    # 예측 결과 추출
    preds = new_raw_predictions['prediction'].squeeze().numpy()
    print(preds)
    # 제출 형식에 맞게 변환
    submission_df = pd.read_csv(DATAROOT+"/sample_submission.csv")
    #submission_df = submission_df[(submission_df['ID'] == 0) | (submission_df['ID'] == 1)]
    date_columns = submission_df.columns[1:]
    submission_df = submission_df.melt(id_vars=['ID'],
                                value_vars=date_columns,
                                var_name='날짜',
                                value_name='값')
    submission_df['날짜'] = pd.to_datetime(submission_df['날짜'])
    submission_df = submission_df.sort_values(['ID', '날짜'])
    submission_df['값'] = preds.flatten()

    return submission_df


In [ ]:
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    submission_df = forecast(p, train)
submission_df

### FORECAST ###


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


KeyboardInterrupt: ignored

In [ ]:
# 날짜 열을 피처로 변환하기 위해 pivot 함수 사용
original_format_df = submission_df.pivot(index='ID', columns='날짜', values='값').reset_index()

# 열 이름을 정확한 날짜 형식으로 변경
original_format_df.columns = ['ID'] + [col.strftime('%Y-%m-%d') for col in original_format_df.columns[1:]]

# 결과 확인
original_format_df.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,3.173858,3.161179,3.137285,3.128660,3.128438,3.129506,3.128668,3.124993,3.120087,...,3.112141,3.111457,3.111162,3.111066,3.111074,3.111127,3.11117,3.111284,3.111366,3.109942
1,1,1.835110,1.792598,1.791018,1.783197,1.781128,1.780619,1.780625,1.780971,1.781263,...,1.781951,1.782160,1.782362,1.782557,1.782751,1.782950,1.78315,1.783304,1.783177,1.783185


In [ ]:
def ensemble(outfn):
    # get all prediction csv files
    fns = list(glob.glob(CSVROOT+"/*.csv"))
    df0 = pd.read_csv(fns[0])
    df = pd.concat([df0] + [pd.read_csv(fn).loc[:,'값'] for fn in fns[1:]], axis=1)
    # get median of all predcitions
    df['median'] = df.iloc[:,1:].median(axis=1)
    df = df[['날짜', 'median']]
    df = df.rename({'median': '값'}, axis=1)
    # save to submission file
    df.to_csv(outfn, index=False)

# not used for final submission
def validate(seed, tr_ds, va_ds):
    va_loader = va_ds.to_dataloader(
        train=False, batch_size=BATCH_SIZE*10, num_workers=12
    )
    best_tft = fit(seed, tr_ds, va_loader)
    actuals = torch.cat([y[0] for x, y in iter(va_loader)])
    predictions = best_tft.predict(va_loader)
    smape_per_num = SMAPE(reduction="none")(predictions, actuals).mean(1)
    print(smape_per_num)
    print(smape_per_num.mean())

In [ ]:
print("### ENSEMBLING ###")
ensemble(CSVROOT + 'submit_v21.csv')

### ENSEMBLING ###


<ipython-input-127-dd76e5d6dd7d>:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['median'] = df.iloc[:,1:].median(axis=1)


In [ ]:
a = pd.read_csv(CSVROOT + 'submit_v21.csv')
a

,num_date_time,answer
0,1_20220825 00,1870.348633
1,1_20220825 01,1756.653076
2,1_20220825 02,1644.968323
3,1_20220825 03,1568.214905
4,1_20220825 04,1601.211426
...,...,...
16795,100_20220831 19,880.414734
16796,100_20220831 20,800.949615
16797,100_20220831 21,730.718262
16798,100_20220831 22,623.565460
